# Import  Library 

In [7]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
import time 
import pandas as pd 
from itertools import permutations


# Get Country Code 

In [19]:
browser = webdriver.Firefox()
browser.get('https://www.trademap.org/Bilateral_TS.aspx?nvpm=1|360||156||TOTAL|||2|1|1|1|2|1|1|1||1')
html_source = browser.page_source
browser.quit()

soup = BeautifulSoup(html_source,'html.parser')
#class "postText" is not defined in the source code
comments = soup.findAll('option')

kode = []
dct= {}
for option in soup.find_all('option'):
    dct[option.text] = option['value']
    kode.append(option['value'])
kode = kode[113:366]

kode_negara = {}
for  key, value in dct.items():
    if value in  kode:
        kode_negara[key] = value
        
kombinasi_negara =  (list(permutations(kode, 2)))

# Get HTML  Importer  Data

In [6]:
def extract_import(negara, partner):

    browser = webdriver.Firefox()
    browser.get(f"https://www.trademap.org/Bilateral_TS.aspx?nvpm=1|{negara}||{partner}||TOTAL|||2|1|1|1|2|1|1|1|1|1")
    time.sleep(3)

    select = Select(browser.find_element_by_id('ctl00_PageContent_GridViewPanelControl_DropDownList_NumTimePeriod'))
    select.select_by_value('5')

    time.sleep(3)
    select = Select(browser.find_element_by_id('ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize'))
    select.select_by_value('300')

    time.sleep(7)
    html_source = browser.page_source

    browser.quit()

    soup = BeautifulSoup(html_source,'html.parser')

    #class "postText" is not defined in the source code
    data_import = soup.findAll('td',{'class':['tabContent']})

    return data_import


# Extract Import Data

In [9]:
def data_import_extract(data_import):
    
    web = []
    for item in data_import:
        tdTags = item.find_all("tr")
    for item in tdTags:
        asi = item.find_all("td")
        for asii in asi :
                #print(no , asii.text)
                #product_code.append(asii.text.strip())
            web.append(asii.text.strip())
    web = web[24:]
    
    return web


In [10]:
def insert_into_list(web):
    m = len(web)
    n =20

    product_code =[]
    product_label = []
    value_2016 =[]
    value_2017 =[]
    value_2018 =[]
    value_2019=[]
    value_2020=[]

    for i in range (0, m, n ):
        #print(web[i])
        product_code.append(web[i])
        product_label.append(web[i+1])
        value_2016.append(web[i+2])
        value_2017.append(web[i+3])
        value_2018.append(web[i+4])
        value_2019.append(web[i+5])
        value_2020.append(web[i+6])
    
    # make dataframe
    kolom = ['product_code', 'product_label','2016','2017','2018','2019','2020']
    df = pd.DataFrame(columns= kolom)
    df['product_code'] = product_code
    df['product_label'] = product_label
    df['2016'] = value_2016
    df['2017'] = value_2017
    df['2018'] = value_2018
    df['2019'] = value_2019
    df['2020'] = value_2020
    return df

In [34]:
def get_key(val):
    for key, value in kode_negara.items():
         if val == value:
            return key

In [ ]:
kolom = ['product_code', 'product_label','2016','2017','2018','2019','2020']
data_importir = pd.DataFrame(columns= kolom)

In [50]:
no = 1
for kombinasi in kombinasi_negara:
    negara = kombinasi[0]
    partner = kombinasi[1]
    nama_negara = get_key(negara)
    nama_partner = get_key(partner)
    data_import = extract_import(negara=negara, partner= partner)
    web = data_import_extract(data_import)
    df = insert_into_list(web)
    df['importer'] = nama_negara  
    df['exporter'] = nama_partner
    data_importir = pd.concat([data_importir, df], ignore_index= True)
    
    
# export to excel
data_importir.to_excel('output.xlsx')
#export to csv
data_importir.to_csv('output.csv')


In [56]:
data_importir

,product_code,product_label,2016,2017,2018,2019,2020,importer,exporter
0,01,Live animals,0,0,0,0,,Afghanistan,Africa not elsewhere specified
1,02,Meat and edible meat offal,0,0,0,0,,Afghanistan,Africa not elsewhere specified
2,03,"Fish and crustaceans, molluscs and other aquat...",0,0,0,0,,Afghanistan,Africa not elsewhere specified
3,04,Dairy produce; birds' eggs; natural honey; edi...,0,0,0,0,,Afghanistan,Africa not elsewhere specified
4,05,"Products of animal origin, not elsewhere speci...",0,0,0,0,,Afghanistan,Africa not elsewhere specified
...,...,...,...,...,...,...,...,...,...
286,94,"Furniture; bedding, mattresses, mattress suppo...",0,0,0,0,,Afghanistan,Albania
287,95,"Toys, games and sports requisites; parts and a...",0,0,0,0,,Afghanistan,Albania
288,96,Miscellaneous manufactured articles,0,0,0,0,,Afghanistan,Albania
289,97,"Works of art, collectors' pieces and antiques",0,0,0,0,,Afghanistan,Albania
